# Hagrid, Provisioning and Deployment

## Materials

The goal for `hagrid` is to be a friendly helper in navigating the world of deployment and provisioning the `syft` stack and to take a multitude of complex steps and turn them into a handful of uniform and consistent commands.

`HAGrid` can deploy to a large variety of targets and by leveraging other tools and formats this can be extended. More specifically, `hagrid` can deploy to:
- `docker`
- any ip (`x.x.x.x`)
- `localhost`
- `azure`
- `gcp`
- `aws` (todo)
- `kubernetes` (coming soon)

### Launch to `Docker`

`hagrid` takes a small number of options and inputs from the user and translates them into lots of terminal commands and network calls as needed to setup and provision a node. In this case, they are `docker` and `docker compose` commands

```docker
docker compose
-p test_domain
--profile vpn
--profile blob-storage
--profile frontend
--env-file /Users/madhavajay/dev/PySyft/packages/grid/.envfile
--file docker-compose.yml
--file docker-compose.build.yml
--file docker-compose.dev.yml
up -d --build
```

In [ ]:
!hagrid launch test_domain domain to docker:8081 --tag=0.7.0 --dev --cmd

### Launch to any IP (`x.x.x.x`)

If we want to deploy `hagrid` to an existing `linux` server somewhere running `ssh`, we can ask `hagrid` to connect to it over `ssh` and do all the setup for us.

The way this works is by leveraging a tool called `ansible` which allows us to define the state in which we want a system to be in, and have it run commands called a `playbook` against that machine until it gets into the desired state.

```bash
!HAGRID_ART=false hagrid launch test_domain domain to 100.0.0.1 \
  --username=ubuntu --auth-type=key \
  --key-path=/Users/madhavajay/.ssh/azureuser.pem \
  --repo=OpenMined/PySyft --branch=0.7.0 --cmd
```

### Launch to `localhost`

All this does is use the same code path as the above `launch to x.x.x.x` except it passes in `--connection=local` which has the effect of telling `ansible` you won't need a username, password or ssh-key, just run locally and use `sudo` as necessary.

```bash
hagrid launch ${7} ${6} to localhost \
    --repo=${2} --branch=${3} \
    --release=${RELEASE} --tag=${DOCKER_TAG}
```

### Launch to Azure

Now that we know how `hagrid` can setup a linux machine for us, let's look at how we might use this to deploy a `cloud vm` onto `azure`.

First, you need to have the `azure-cli` tools which have many python packages 

```bash
hagrid launch test_domain domain to azure
```

### Launch to GCP

```bash
hagrid launch test_domain domain to gcp 
```

## Trial of the Flesh

#### 1. Deploy the `network` node either on `docker` or another `vm` somewhere on on-prem infrastructure or the `cloud`


Launching the `network` to `docker`

```bash
hagrid launch test_network network to docker:8082 --tag=0.7.0 --tail --dev
```

Check the `network` health

In [8]:
import syft as sy
import hagrid

hagrid.check('localhost:8082')

┏━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━┓
┃ PyGrid    ┃ Info                                      ┃    ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━┩
│ UI (βeta) │ http://localhost:8082/login               │ ✅ │
│ api       │ http://localhost:8082/api/v1/openapi.json │ ✅ │
└───────────┴───────────────────────────────────────────┴────┘

Log in to the `network` and check its initial `domains`

In [9]:
network = sy.login('localhost', 8082)
network.domains

Connecting to localhost... done! 	 Logging into test_network... as GUEST...done!


""


We can see that there is no connected `domain` yet

#### 2. Launch 2 `domain` nodes on other systems


##### Launch the first `domain` to a remote machine:
1. Connect to the `narvi` machine with `ssh -i narvi_key nguyen29@narvi.tut.fi`
2. Find out the `ip` address of the narvi node with `ip route`. Let's say we found out that the `ip address` is `130.230.56.40`
3. Launch the `domain` to the `narvi` node with
```bash
HAGRID_ART=false hagrid launch test_domain domain to 130.230.56.40 \
  --username=nguyen29 --auth-type=key \
  --key-path=/home/dk/Desktop/projects/ssh_keys/narvi_key \
  --repo=OpenMined/PySyft --branch=0.7.0
```
we will see something like below running:
![](./assets/5-hagrid-launch_to_ip.png)

Check the `domain` health

In [15]:
hagrid.check('130.230.56.40:80')

Output()

✅ test_network Network Containers Created

✅ Backend

✅ Startup Complete

┏━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━┓
┃ PyGrid    ┃ Info                                        ┃ 600 ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━┩
│ host      │ 130.230.56.40:80                            │ ❌  │
│ UI (βeta) │ http://130.230.56.40:80/login               │ ❌  │
│ api       │ http://130.230.56.40:80/api/v1/openapi.json │ ❌  │
│ ssh       │ hagrid ssh 130.230.56.40:80                 │ ❌  │
│ jupyter   │ http://130.230.56.40:80:8888                │ ❌  │
└───────────┴─────────────────────────────────────────────┴─────┘

In [ ]:
domain_client_1 = sy.login(
    url="130.230.56.40",
    port=80,
    email="info@openmined.org",
    password="changethis"
)

##### Launch the second `domain` to

In [ ]:
domain_client_2

#### 3. Connect each `domain` to the `network` and verify that they can see each other. They are connected via `tailscale`

In [ ]:
domain_client_1.apply_to_network(network)

In [ ]:
domain_client_2.apply_to_network(network)

In [13]:
network.domains

""
